# 앱스토어 댓글 요약 RAG 시스템
## Ollama 환경에서 동작하는 댓글 분석 및 요약 기능

### 주요 기능:
- CSV 파일에서 앱스토어 댓글 로드
- RAG 기반 댓글 임베딩 및 검색
- 유사한 댓글 그룹핑
- 자동 요약 생성
- 감정 분석 및 키워드 추출


In [ ]:
# 필요한 라이브러리 설치 및 import
import os, re, json, csv, pandas as pd
from typing import List, Dict, Any, Tuple, Optional, Set
from dataclasses import dataclass
from collections import Counter, defaultdict
import numpy as np

from langchain.docstore.document import Document
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# 환경 설정
EMBED_MODEL = "bge-m3:latest"
LLM_MODEL = "exaone3.5:latest"
FAISS_DIR = "models/faiss_comment_summary"

print("앱스토어 댓글 요약 RAG 시스템 초기화")
print(f"임베딩 모델: {EMBED_MODEL}")
print(f"언어 모델: {LLM_MODEL}")


In [ ]:
# 댓글 전처리 및 분석 클래스
class CommentProcessor:
    def __init__(self):
        self.positive_keywords = [
            '좋다', '좋은', '최고', '만족', '재미', '훌륭', '완벽', '추천', '성공', '편리', '쉽다', 
            '빠르다', '안정', '깔끔', '예쁘다', '멋지다', '감동', '놀랍', '뛰어나다', '우수', '괜찮'
        ]
        self.negative_keywords = [
            '나쁘다', '나쁜', '최악', '불만', '짜증', '화나다', '실망', '엉망', '느리다', '불편', 
            '어렵다', '복잡', '불안정', '버그', '오류', '문제', '고장', '작동안됨', '끊김', '렉'
        ]
        self.game_keywords = [
            '게임', '플레이', '캐릭터', '스킬', '레벨', '아이템', '퀘스트', '던전', '길드', '팀', 
            '대전', '랭킹', '점수', '보상', '이벤트', '업데이트', '패치', '밸런스', '과금', '유료'
        ]
        
    def clean_comment(self, comment: str) -> str:
        """댓글 텍스트 정리"""
        # 특수문자 및 이모지 제거 (일부 보존)
        comment = re.sub(r'[^\w\s가-힣ㄱ-ㅎㅏ-ㅣ.!?\-]', ' ', comment)
        # 연속된 공백 제거
        comment = re.sub(r'\s+', ' ', comment).strip()
        return comment
    
    def extract_sentiment(self, comment: str) -> str:
        """간단한 감정 분석"""
        positive_count = sum(1 for keyword in self.positive_keywords if keyword in comment)
        negative_count = sum(1 for keyword in self.negative_keywords if keyword in comment)
        
        if positive_count > negative_count:
            return "긍정"
        elif negative_count > positive_count:
            return "부정"
        else:
            return "중립"
    
    def extract_keywords(self, comment: str) -> List[str]:
        """댓글에서 키워드 추출"""
        keywords = []
        
        # 게임 관련 키워드
        for keyword in self.game_keywords:
            if keyword in comment:
                keywords.append(keyword)
        
        # 기본적인 명사 추출 (간단한 패턴)
        # 실제로는 형태소 분석기를 사용하는 것이 좋음
        nouns = re.findall(r'[가-힣]{2,}', comment)
        keywords.extend([noun for noun in nouns if len(noun) >= 2 and len(noun) <= 6])
        
        return list(set(keywords))
    
    def process_comment(self, comment: str) -> Dict[str, Any]:
        """댓글 종합 분석"""
        cleaned = self.clean_comment(comment)
        sentiment = self.extract_sentiment(cleaned)
        keywords = self.extract_keywords(cleaned)
        
        return {
            'original': comment,
            'cleaned': cleaned,
            'sentiment': sentiment,
            'keywords': keywords,
            'length': len(cleaned)
        }

print("댓글 처리 클래스 정의 완료")


In [ ]:
# CSV 데이터 로더
def load_comments_from_csv(csv_path: str, comment_column: str = 'comment', 
                          rating_column: str = 'rating', limit: int = None) -> List[Dict]:
    """CSV 파일에서 댓글 데이터 로드"""
    comments = []
    processor = CommentProcessor()
    
    try:
        df = pd.read_csv(csv_path, encoding='utf-8')
        print(f"CSV 파일 로드 완료: {len(df)} 개 행")
        
        # 컬럼 확인
        if comment_column not in df.columns:
            print(f"경고: '{comment_column}' 컬럼을 찾을 수 없음. 사용 가능한 컬럼: {list(df.columns)}")
            return []
        
        for idx, row in df.iterrows():
            if limit and idx >= limit:
                break
                
            comment_text = str(row[comment_column]).strip()
            if len(comment_text) < 10:  # 너무 짧은 댓글 제외
                continue
                
            processed = processor.process_comment(comment_text)
            processed['id'] = idx
            processed['rating'] = row.get(rating_column, None)
            
            comments.append(processed)
            
        print(f"처리된 댓글 수: {len(comments)}")
        return comments
        
    except Exception as e:
        print(f"CSV 로드 오류: {e}")
        return []

# 샘플 CSV 생성 함수
def create_sample_csv(file_path: str = "sample_comments.csv"):
    """테스트용 샘플 CSV 파일 생성"""
    sample_comments = [
        {"comment": "그래픽이 정말 좋고 타격감도 훌륭합니다. 과금 없이도 충분히 재미있어요!", "rating": 5},
        {"comment": "MMORPG 치고는 케주얼하게 즐길 수 있어서 좋네요. 업데이트가 좀 더 자주 있었으면 좋겠어요", "rating": 4},
        {"comment": "다른 게임의 아류작 같은 느낌이에요. 그래도 타격감은 괜찮습니다", "rating": 3},
        {"comment": "레벨업이 너무 느려요. 과금 유도가 심한 것 같습니다", "rating": 2},
        {"comment": "버그가 너무 많아요. 게임이 자주 튕기고 렉도 심합니다", "rating": 1},
        {"comment": "캐릭터 디자인이 예쁘고 스킬 이펙트도 화려해요. 추천합니다!", "rating": 5},
        {"comment": "던전 시스템이 재미있고 길드 활동도 활발해서 좋아요", "rating": 4},
        {"comment": "초보자도 쉽게 시작할 수 있어서 좋습니다. UI도 직관적이에요", "rating": 4},
        {"comment": "이벤트 보상이 짜요. 더 많은 혜택이 있었으면 좋겠네요", "rating": 3},
        {"comment": "PvP 밸런스가 엉망이에요. 과금러들만 유리한 구조입니다", "rating": 2},
        {"comment": "음악과 사운드 이펙트가 정말 좋아요. 몰입도가 높습니다", "rating": 5},
        {"comment": "퀘스트가 다양하고 스토리도 흥미진진해요", "rating": 4},
        {"comment": "서버 안정성이 떨어져요. 접속이 자주 끊깁니다", "rating": 2},
        {"comment": "아이템 강화 확률이 너무 낮아요. 돈이 너무 많이 듭니다", "rating": 2},
        {"comment": "전체적으로 완성도가 높은 게임이에요. 시간 가는 줄 모르겠습니다", "rating": 5}
    ]
    
    df = pd.DataFrame(sample_comments)
    df.to_csv(file_path, index=False, encoding='utf-8')
    print(f"샘플 CSV 파일 생성: {file_path}")
    return file_path

print("CSV 로더 함수 정의 완료")


In [ ]:
# RAG 기반 댓글 요약 시스템
class CommentSummaryRAG:
    def __init__(self, embed_model: str = EMBED_MODEL, llm_model: str = LLM_MODEL):
        self.embed_model = embed_model
        self.llm_model = llm_model
        self.embeddings = OllamaEmbeddings(model=embed_model)
        self.llm = ChatOllama(model=llm_model, temperature=0.1)
        self.vectorstore = None
        self.retriever = None
        self.comments = []
        
    def prepare_documents(self, comments: List[Dict]) -> List[Document]:
        """댓글을 Document 객체로 변환"""
        documents = []
        
        for comment in comments:
            # 메타데이터에 분석 결과 포함
            metadata = {
                'id': comment['id'],
                'sentiment': comment['sentiment'],
                'keywords': ', '.join(comment['keywords'][:5]),  # 상위 5개 키워드만
                'rating': comment.get('rating', None),
                'length': comment['length']
            }
            
            doc = Document(
                page_content=comment['cleaned'],
                metadata=metadata
            )
            documents.append(doc)
            
        return documents
    
    def build_vectorstore(self, comments: List[Dict]):
        """벡터 스토어 구축"""
        print("벡터 스토어 구축 중...")
        self.comments = comments
        documents = self.prepare_documents(comments)
        
        # FAISS 벡터 스토어 생성
        self.vectorstore = FAISS.from_documents(documents, self.embeddings)
        
        # BM25 검색기 생성
        bm25_retriever = BM25Retriever.from_documents(documents)
        bm25_retriever.k = 5
        
        # 하이브리드 검색기 (FAISS + BM25)
        faiss_retriever = self.vectorstore.as_retriever(search_kwargs={"k": 5})
        self.retriever = EnsembleRetriever(
            retrievers=[faiss_retriever, bm25_retriever],
            weights=[0.7, 0.3]
        )
        
        print(f"벡터 스토어 구축 완료: {len(documents)}개 댓글")
    
    def cluster_comments(self, n_clusters: int = 5) -> Dict[int, List[Dict]]:
        """댓글을 주제별로 클러스터링"""
        if not self.vectorstore:
            print("벡터 스토어가 구축되지 않았습니다.")
            return {}
        
        print(f"댓글 클러스터링 중... (클러스터 수: {n_clusters})")
        
        # 모든 댓글의 임베딩 가져오기
        texts = [comment['cleaned'] for comment in self.comments]
        embeddings = self.embeddings.embed_documents(texts)
        embeddings_array = np.array(embeddings)
        
        # K-means 클러스터링
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(embeddings_array)
        
        # 클러스터별로 댓글 그룹화
        clusters = defaultdict(list)
        for i, label in enumerate(cluster_labels):
            clusters[label].append(self.comments[i])
        
        print(f"클러스터링 완료: {len(clusters)}개 그룹")
        return dict(clusters)

print("CommentSummaryRAG 클래스 (1/2) 정의 완료")


In [ ]:
# CommentSummaryRAG 클래스 계속 (요약 기능)
def add_summary_methods():
    def generate_cluster_summary(self, cluster_comments: List[Dict]) -> str:
        """클러스터의 댓글들을 요약"""
        if not cluster_comments:
            return "댓글이 없습니다."
        
        # 댓글 텍스트들을 결합
        comment_texts = [comment['cleaned'] for comment in cluster_comments]
        combined_text = '\n'.join(comment_texts[:10])  # 최대 10개 댓글만 사용
        
        # 감정 분포 계산
        sentiments = [comment['sentiment'] for comment in cluster_comments]
        sentiment_counts = Counter(sentiments)
        
        # 평점 분포 계산
        ratings = [comment['rating'] for comment in cluster_comments if comment['rating']]
        avg_rating = sum(ratings) / len(ratings) if ratings else 0
        
        # 키워드 추출
        all_keywords = []
        for comment in cluster_comments:
            all_keywords.extend(comment['keywords'])
        top_keywords = [word for word, count in Counter(all_keywords).most_common(5)]
        
        # 요약 프롬프트
        prompt = PromptTemplate.from_template(
            """다음은 앱스토어에서 수집된 게임 리뷰 댓글들입니다. 
이 댓글들의 주요 내용을 3-4문장으로 요약해주세요.

댓글들:
{comments}

추가 정보:
- 감정 분포: {sentiment_dist}
- 평균 평점: {avg_rating:.1f}
- 주요 키워드: {keywords}

요약 (간결하고 핵심적인 내용으로):"""
        )
        
        chain = prompt | self.llm | StrOutputParser()
        
        try:
            summary = chain.invoke({
                "comments": combined_text,
                "sentiment_dist": dict(sentiment_counts),
                "avg_rating": avg_rating,
                "keywords": ', '.join(top_keywords)
            })
            return summary
        except Exception as e:
            print(f"요약 생성 오류: {e}")
            return f"자동 요약: 총 {len(cluster_comments)}개 댓글, 주요 키워드: {', '.join(top_keywords[:3])}"
    
    def generate_overall_summary(self) -> str:
        """전체 댓글에 대한 종합 요약"""
        if not self.comments:
            return "분석할 댓글이 없습니다."
        
        # 전체 통계
        total_comments = len(self.comments)
        sentiments = Counter([c['sentiment'] for c in self.comments])
        ratings = [c['rating'] for c in self.comments if c['rating']]
        avg_rating = sum(ratings) / len(ratings) if ratings else 0
        
        # 상위 키워드
        all_keywords = []
        for comment in self.comments:
            all_keywords.extend(comment['keywords'])
        top_keywords = [word for word, count in Counter(all_keywords).most_common(10)]
        
        # 대표적인 긍정/부정 댓글
        positive_comments = [c for c in self.comments if c['sentiment'] == '긍정'][:3]
        negative_comments = [c for c in self.comments if c['sentiment'] == '부정'][:3]
        
        positive_text = '\n'.join([c['cleaned'] for c in positive_comments])
        negative_text = '\n'.join([c['cleaned'] for c in negative_comments])
        
        prompt = PromptTemplate.from_template(
            """다음은 앱스토어 게임 리뷰 댓글 분석 결과입니다.
전체적인 사용자 반응을 종합하여 요약해주세요.

통계 정보:
- 총 댓글 수: {total_comments}
- 감정 분포: 긍정 {positive}개, 부정 {negative}개, 중립 {neutral}개
- 평균 평점: {avg_rating:.1f}/5.0
- 주요 키워드: {keywords}

대표 긍정 댓글:
{positive_comments}

대표 부정 댓글:
{negative_comments}

종합 요약 (게임의 장단점을 균형있게 포함):"""
        )
        
        chain = prompt | self.llm | StrOutputParser()
        
        try:
            summary = chain.invoke({
                "total_comments": total_comments,
                "positive": sentiments['긍정'],
                "negative": sentiments['부정'],
                "neutral": sentiments['중립'],
                "avg_rating": avg_rating,
                "keywords": ', '.join(top_keywords),
                "positive_comments": positive_text,
                "negative_comments": negative_text
            })
            return summary
        except Exception as e:
            print(f"종합 요약 생성 오류: {e}")
            return f"총 {total_comments}개 댓글 분석 완료 (평균 평점: {avg_rating:.1f})"
    
    # 메소드 추가
    CommentSummaryRAG.generate_cluster_summary = generate_cluster_summary
    CommentSummaryRAG.generate_overall_summary = generate_overall_summary

add_summary_methods()
print("CommentSummaryRAG 클래스 (2/2) 요약 기능 추가 완료")


In [ ]:
# 메인 실행 코드
def main():
    # 1. 샘플 CSV 파일 생성 (테스트용)
    print("=== 1. 샘플 데이터 생성 ===")
    csv_file = create_sample_csv("sample_comments.csv")
    
    # 2. 댓글 데이터 로드
    print("\n=== 2. 댓글 데이터 로드 ===")
    comments = load_comments_from_csv(csv_file)
    
    if not comments:
        print("댓글 데이터를 불러올 수 없습니다.")
        return
    
    # 3. RAG 시스템 초기화
    print("\n=== 3. RAG 시스템 초기화 ===")
    rag_system = CommentSummaryRAG()
    
    # 4. 벡터 스토어 구축
    print("\n=== 4. 벡터 스토어 구축 ===")
    rag_system.build_vectorstore(comments)
    
    # 5. 댓글 클러스터링
    print("\n=== 5. 댓글 클러스터링 ===")
    clusters = rag_system.cluster_comments(n_clusters=3)
    
    # 6. 클러스터별 요약
    print("\n=== 6. 클러스터별 요약 ===")
    for cluster_id, cluster_comments in clusters.items():
        print(f"\n--- 클러스터 {cluster_id + 1} ({len(cluster_comments)}개 댓글) ---")
        
        # 클러스터 내 댓글 미리보기
        print("대표 댓글:")
        for i, comment in enumerate(cluster_comments[:2]):
            print(f"  {i+1}. {comment['cleaned'][:50]}...")
        
        # 클러스터 요약
        summary = rag_system.generate_cluster_summary(cluster_comments)
        print(f"\n요약: {summary}")
    
    # 7. 전체 종합 요약
    print("\n=== 7. 전체 종합 요약 ===")
    overall_summary = rag_system.generate_overall_summary()
    print(f"\n종합 요약:\n{overall_summary}")
    
    # 8. 통계 정보
    print("\n=== 8. 통계 정보 ===")
    sentiments = Counter([c['sentiment'] for c in comments])
    print(f"감정 분포: {dict(sentiments)}")
    
    ratings = [c['rating'] for c in comments if c['rating']]
    if ratings:
        print(f"평점 분포: 평균 {sum(ratings)/len(ratings):.1f}, 최고 {max(ratings)}, 최저 {min(ratings)}")
    
    # 키워드 빈도
    all_keywords = []
    for comment in comments:
        all_keywords.extend(comment['keywords'])
    top_keywords = Counter(all_keywords).most_common(10)
    print(f"상위 키워드: {top_keywords}")

# 실행
print("main() 함수 정의 완료. 실행하려면 다음 셀에서 main()을 호출하세요.")


In [ ]:
# 추가 기능: 특정 주제나 키워드로 댓글 검색
def search_comments_by_topic(rag_system: CommentSummaryRAG, query: str, top_k: int = 5):
    """특정 주제나 키워드로 관련 댓글 검색"""
    if not rag_system.retriever:
        print("RAG 시스템이 초기화되지 않았습니다.")
        return []
    
    print(f"검색 쿼리: '{query}'")
    
    # 관련 댓글 검색
    relevant_docs = rag_system.retriever.invoke(query)
    
    print(f"\n검색 결과 ({len(relevant_docs)}개):")
    for i, doc in enumerate(relevant_docs[:top_k]):
        print(f"\n{i+1}. {doc.page_content}")
        print(f"   메타데이터: {doc.metadata}")
    
    return relevant_docs[:top_k]

# 실제 CSV 파일 사용 예시
def analyze_real_csv(csv_path: str, comment_col: str = 'comment', rating_col: str = 'rating'):
    """실제 CSV 파일로 분석 실행"""
    print(f"실제 CSV 파일 분석: {csv_path}")
    
    # 댓글 로드
    comments = load_comments_from_csv(csv_path, comment_col, rating_col, limit=100)  # 처음 100개만
    
    if not comments:
        print("댓글을 불러올 수 없습니다.")
        return
    
    # RAG 시스템 구축 및 분석
    rag_system = CommentSummaryRAG()
    rag_system.build_vectorstore(comments)
    
    # 클러스터링 및 요약
    clusters = rag_system.cluster_comments(n_clusters=3)
    
    # 결과 출력
    for cluster_id, cluster_comments in clusters.items():
        print(f"\n=== 그룹 {cluster_id + 1} ===\n")
        summary = rag_system.generate_cluster_summary(cluster_comments)
        print(summary)
    
    # 전체 요약
    print("\n=== 전체 요약 ===\n")
    overall_summary = rag_system.generate_overall_summary()
    print(overall_summary)
    
    return rag_system

print("추가 기능 함수들 정의 완료")


## 사용 방법

### 1. 기본 실행 (샘플 데이터)
```python
# 샘플 데이터로 테스트 실행
main()
```

### 2. 실제 CSV 파일 사용
```python
# 실제 CSV 파일 분석 (컬럼명 수정 필요)
rag_system = analyze_real_csv('your_comments.csv', 'comment_text', 'star_rating')
```

### 3. 특정 주제로 댓글 검색
```python
# RAG 시스템이 구축된 후
search_comments_by_topic(rag_system, "그래픽이 좋다")
search_comments_by_topic(rag_system, "버그가 많다")
search_comments_by_topic(rag_system, "과금")
```

### 주의사항:
- **Ollama 설치**: `bge-m3:latest`와 `exaone3.5:latest` 모델이 설치되어 있어야 함
- **CSV 컬럼**: 댓글과 평점 컬럼명을 정확히 지정해야 함
- **대용량 데이터**: `limit` 매개변수로 개수 제한 권장
- **한국어 최적화**: 한국어 댓글에 특화되어 있음


In [ ]:
# 실행하기
# 아래 주석을 해제하고 실행해보세요!

# 기본 샘플 데이터로 테스트
main()

# 실제 CSV 파일이 있다면:
# rag_system = analyze_real_csv('path/to/your/comments.csv', 'comment', 'rating')

# 특정 키워드로 검색하려면:
# search_comments_by_topic(rag_system, "원하는 키워드")
